# Data Preparation and Padding

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np

class ThermalHistoryDataset(Dataset):
    def __init__(self, input_dir, output_dir, window_size=1000, step_size=500, scaler=None):  # Decreased window size
        self.input_files = {os.path.basename(f): os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.txt')}
        self.output_files = {os.path.basename(f): os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.txt')}
        
        assert self.input_files.keys() == self.output_files.keys(), "Mismatch in input and output file names."
        
        self.file_names = sorted(self.input_files.keys())
        self.scaler = scaler or StandardScaler()
        self.window_size = window_size
        self.step_size = step_size
        self._fit_scaler()
        print(f"Initialized dataset with {len(self.file_names)} files.")

    def _fit_scaler(self):
        all_input_features = []
        for file_name in self.file_names:
            data = pd.read_csv(self.input_files[file_name], delimiter=',')[[ 'tm', 'tl', 'cr']].values
            all_input_features.append(data)
        all_input_features = np.vstack(all_input_features)
        self.scaler.fit(all_input_features)
        print("Scaler fitted to the data.")

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        input_data = pd.read_csv(self.input_files[file_name], delimiter=',')
        output_data = pd.read_csv(self.output_files[file_name], delimiter=',')
        
        input_coords = input_data[['x', 'y', 'z']].values
        input_features = input_data[['tm', 'tl', 'cr']].values
        output_features = output_data[['tm', 'tl', 'cr']].values
        
        input_features = self.scaler.transform(input_features)
        output_features = self.scaler.transform(output_features)

        input_features = np.concatenate((input_coords, input_features), axis=1)
        output_features = np.concatenate((input_coords, output_features), axis=1)

        input_windows = []
        output_windows = []

        for start in range(0, len(input_features) - self.window_size + 1, self.step_size):
            end = start + self.window_size
            input_windows.append(input_features[start:end])
            output_windows.append(output_features[start:end])

        input_windows = np.array(input_windows)
        output_windows = np.array(output_windows)
        
        if idx % 10 == 0:
            print(f"Processed file {idx + 1}/{len(self.file_names)}")

        return {
            'input_windows': torch.tensor(input_windows, dtype=torch.float),
            'output_windows': torch.tensor(output_windows, dtype=torch.float)
        }

def collate_fn(batch):
    input_windows = [item['input_windows'] for item in batch]
    output_windows = [item['output_windows'] for item in batch]

    input_windows = torch.cat(input_windows, dim=0)
    output_windows = torch.cat(output_windows, dim=0)

    return {
        'input_windows': input_windows,
        'output_windows': output_windows
    }

from torch.utils.data import DataLoader

# Create dataset with sliding window parameters
window_size = 1000  # Reduced window size
step_size = 500  # Adjust step size if needed
dataset = ThermalHistoryDataset(input_dir='./../../data/input', output_dir='./../../data/output', window_size=window_size, step_size=step_size)

# Create dataloader with custom collate function
batch_size = 2  # Reduced batch size
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


# Transformer Model

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):  # Adjusted max_len to a reasonable value
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        seq_len = x.size(1)
        return x + self.encoding[:, :seq_len, :]

class TransformerModel(nn.Module):
    def __init__(self, feature_size, num_layers, num_heads, dim_feedforward, dropout_rate=0.1):
        super(TransformerModel, self).__init__()
        self.positional_encoding = PositionalEncoding(feature_size, max_len=5000)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=feature_size,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            batch_first=True,
            dropout=dropout_rate
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.decoder = nn.Sequential(
            nn.Linear(feature_size, feature_size),
            nn.ReLU(),
            nn.Linear(feature_size, feature_size)
        )

    def forward(self, src):
        src = self.positional_encoding(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

# Initialize the model
feature_size = 6  # Adjusted to match the concatenated feature size (coords + features)
num_heads = 2
num_layers = 4
dim_feedforward = 256
dropout_rate = 0.1
model = TransformerModel(feature_size=feature_size, num_layers=num_layers, num_heads=num_heads, dim_feedforward=dim_feedforward, dropout_rate=dropout_rate)


# Training and Validation

In [ ]:
import torch.optim as optim

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss Function
criterion = torch.nn.MSELoss()

# Training Function
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_windows = batch['input_windows']
        output_windows = batch['output_windows']
        
        # Forward pass
        optimizer.zero_grad()
        predictions = model(input_windows)
        
        # Calculate loss
        loss = criterion(predictions, output_windows)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# Validation Function remains the same but includes device handling
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_windows = batch['input_windows']
            output_windows = batch['output_windows']
            
            predictions = model(input_windows)
            loss = criterion(predictions, output_windows)
            total_loss += loss.item()
            
    return total_loss / len(dataloader)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, dataloader, optimizer, criterion)
    # val_loss = validate(model, dataloader, criterion)  # Assuming val_dataloader is defined
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}")
